# S&P 100 Data Estimates - Ex-Post Returns

In [123]:
from __future__ import print_function
import pandas as pd
import numpy as np
import datetime as dt
import quandl

config = {'data': {'start_date': dt.date(2010, 1, 1),
                   'end_date': dt.date(2018, 12, 31),
                   'authtoken': "6XyApK2BBj_MraQg2TMD",
                   'collapse': 'monthly'},
          'returns': {'mode': 'expost',
                      'horizon': 1,
                      'lambda': 0.1},
          'universe': {'risk_free_symbol': 'USDOLLAR'},
          'risk': 'FF'
         }

## Data 

In [94]:
datadir='../../data/'
assets=pd.read_csv(datadir + 'SP100.csv', comment='#').set_index('Symbol')
data={}

In [90]:
from datetime import datetime
import pandas_datareader.data as web
ds = web.DataReader('North_America_5_Factors_Daily', 'famafrench',
                    start=config['data']['start_date'], end=config['data']['end_date'])
ff_returns = ds[0]
ff_returns.index = ff_returns.index.to_timestamp()
ff_returns

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2010-01-01,0.05,0.03,0.04,-0.03,-0.06,0.00
2010-01-04,1.69,0.76,0.87,0.12,0.08,0.00
2010-01-05,0.31,-0.09,0.73,-0.05,-0.12,0.00
2010-01-06,0.23,0.13,0.69,-0.06,0.01,0.00
2010-01-07,0.28,0.28,0.71,-0.64,0.27,0.00
2010-01-08,0.42,0.29,0.21,0.18,-0.45,0.00
2010-01-11,0.07,-0.08,-0.25,-0.02,0.54,0.00
2010-01-12,-1.06,-0.43,-0.92,0.41,0.18,0.00
2010-01-13,0.91,0.32,0.29,-0.25,0.00,0.00


#### Download loop

If it stops because of Quandl error codes 503 or 504, try re-running it (it won't download data already downloaded). If Quandl complains about the speed of requests, try adding sleep time.

In [95]:
# Download assets' data
def to_quandl_eod_ticker(ticker):
    '''
    Converts ticker to format in Quandl EOD dataset
    '''
    if 'USDOLLAR' not in ticker:
        return 'EOD/' + ticker.replace('.', '_')
    else:
        return 'FRED/DTB3'

# Construct a data dictionary: {ticker: pd.DataFrame(price/volume)}
for ticker in assets.index:
    if ticker in data:
        continue
    print('downloading %s from %s to %s' %(ticker, config['data']['start_date'], config['data']['end_date']))
    try:
        data[ticker] = quandl.get(to_quandl_eod_ticker(ticker), **config['data'])
    except quandl.NotFoundError:
        print('\tInvalid asset code')

downloading AAPL from 2010-01-01 to 2018-12-31
downloading ABBV from 2010-01-01 to 2018-12-31
downloading ABT from 2010-01-01 to 2018-12-31
downloading ACN from 2010-01-01 to 2018-12-31
downloading ADBE from 2010-01-01 to 2018-12-31
downloading AGN from 2010-01-01 to 2018-12-31
downloading AIG from 2010-01-01 to 2018-12-31
downloading ALL from 2010-01-01 to 2018-12-31
downloading AMGN from 2010-01-01 to 2018-12-31
downloading AMZN from 2010-01-01 to 2018-12-31
downloading AXP from 2010-01-01 to 2018-12-31
downloading BA from 2010-01-01 to 2018-12-31
downloading BAC from 2010-01-01 to 2018-12-31
downloading BIIB from 2010-01-01 to 2018-12-31
downloading BK from 2010-01-01 to 2018-12-31
downloading BKNG from 2010-01-01 to 2018-12-31
downloading BLK from 2010-01-01 to 2018-12-31
downloading BMY from 2010-01-01 to 2018-12-31
downloading BRK.B from 2010-01-01 to 2018-12-31
downloading C from 2010-01-01 to 2018-12-31
downloading CAT from 2010-01-01 to 2018-12-31
downloading CELG from 2010-01

#### Computation 

In [99]:
keys=[el for el in assets.index if not el in (set(assets.index)-set(data.keys()))]

def select_first_valid_column(df, columns):
    for column in columns:
        if column in df.columns:
            return df[column]

# extract prices
prices=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Adj. Close", "Close", "Value"])
                                              for k in keys])))

#compute sigmas
open_price=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Open"]) for k in keys])))
close_price=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Close"]) for k in keys])))
sigmas = np.abs(np.log(open_price.astype(float))-np.log(close_price.astype(float)))

# extract volumes
volumes=pd.DataFrame.from_dict(dict(zip(keys, [select_first_valid_column(data[k], ["Adj. Volume", "Volume"])
                                               for k in keys])))

# fix risk free
prices[config['universe']['risk_free_symbol']]=10000*(1 + prices[config['universe']['risk_free_symbol']]/(100*250)).cumprod()

#### Filtering 

In [100]:
# filter NaNs - threshold at 2% missing values
bad_assets = prices.columns[prices.isnull().sum()>len(prices)*0.02]
if len(bad_assets):
    print('Assets %s have too many NaNs, removing them' % bad_assets)

prices = prices.loc[:,~prices.columns.isin(bad_assets)]
sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]

nassets=prices.shape[1]

# days on which many assets have missing values
bad_days1=sigmas.index[sigmas.isnull().sum(1) > nassets*.9]
bad_days2=prices.index[prices.isnull().sum(1) > nassets*.9]
bad_days3=volumes.index[volumes.isnull().sum(1) > nassets*.9]
bad_days=pd.Index(set(bad_days1).union(set(bad_days2)).union(set(bad_days3))).sort_values()
print ("Removing these days from dataset:")
print(pd.DataFrame({'nan price':prices.isnull().sum(1)[bad_days],
                    'nan volumes':volumes.isnull().sum(1)[bad_days],
                    'nan sigmas':sigmas.isnull().sum(1)[bad_days]}))

prices=prices.loc[~prices.index.isin(bad_days)]
sigmas=sigmas.loc[~sigmas.index.isin(bad_days)]
volumes=volumes.loc[~volumes.index.isin(bad_days)]

# extra filtering
print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
                    'remaining nan volumes':volumes.isnull().sum(),
                    'remaining nan sigmas':sigmas.isnull().sum()}))
prices=prices.fillna(method='ffill')
sigmas=sigmas.fillna(method='ffill')
volumes=volumes.fillna(method='ffill')
print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
                    'remaining nan volumes':volumes.isnull().sum(),
                    'remaining nan sigmas':sigmas.isnull().sum()}))

Assets Index(['ABBV', 'DD', 'DOW', 'FB', 'GM', 'GOOG', 'KHC', 'KMI', 'PYPL'], dtype='object') have too many NaNs, removing them
Removing these days from dataset:
Empty DataFrame
Columns: [nan price, nan volumes, nan sigmas]
Index: []
          remaining nan price  remaining nan volumes  remaining nan sigmas
AAPL                        0                      0                     0
ABT                         0                      0                     0
ACN                         0                      0                     0
ADBE                        0                      0                     0
AGN                         0                      0                     0
...                       ...                    ...                   ...
WBA                         0                      0                     0
WFC                         0                      0                     0
WMT                         0                      0                     0
XOM             

#### Save 

In [101]:
# make volumes in dollars
volumes = volumes*prices

# compute returns
returns = (prices.diff()/prices.shift(1)).fillna(method='ffill').iloc[1:]

bad_assets = returns.columns[((-.5>returns).sum()>0)|((returns > 2.).sum()>0)]
if len(bad_assets):
    print('Assets %s have dubious returns, removed' % bad_assets)
    
prices = prices.loc[:,~prices.columns.isin(bad_assets)]
sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]
returns = returns.loc[:,~returns.columns.isin(bad_assets)]

# remove USDOLLAR except from returns
prices = prices.iloc[:,:-1]
sigmas = sigmas.iloc[:,:-1]
volumes = volumes.iloc[:,:-1]


# save data
prices.to_csv(datadir+'rex_prices.csv.gz', compression='gzip', float_format='%.3f')
volumes.to_csv(datadir+'rex_volumes.csv.gz', compression='gzip', float_format='%d')
returns.to_csv(datadir+'rex_returns.csv.gz', compression='gzip', float_format='%.3e')
sigmas.to_csv(datadir+'rex_sigmas.csv.gz', compression='gzip', float_format='%.3e')

Assets Index(['AAPL', 'C', 'CL', 'CMCSA', 'DHR', 'GOOGL', 'KO', 'MA', 'NFLX', 'NKE',
       'V'],
      dtype='object') have dubious returns, removed


## Estimates 

In [121]:
import numpy as np

print("Typical variance of returns: %g"%returns.var().mean())

Typical variance of returns: 0.00457975


In [122]:
returns

,ABT,ACN,ADBE,AGN,AIG,ALL,AMGN,AMZN,AXP,BA,...,UNP,UPS,USB,UTX,VZ,WBA,WFC,WMT,XOM,USDOLLAR
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-28,0.025312,-0.024884,0.072755,0.037008,0.022286,0.044103,-0.031977,-0.055897,0.014073,0.042244,...,0.113554,0.016791,-0.018740,0.017338,-0.016655,-0.022469,-0.038340,0.011978,0.008847,5.200000e-06
2010-03-31,-0.029477,0.049537,0.020779,0.049761,0.378280,0.033920,0.057013,0.146706,0.080388,0.149620,...,0.088021,0.096527,0.051605,0.072251,0.072243,0.052497,0.138259,0.028297,0.030462,6.400000e-06
2010-04-30,-0.028853,0.040286,-0.050042,0.025138,0.139426,0.011142,-0.042239,0.009796,0.117790,-0.002479,...,0.032196,0.073436,0.034389,0.018204,-0.068343,-0.052305,0.063946,-0.035252,0.011795,6.400000e-06
2010-05-31,-0.070367,-0.140238,-0.045238,0.031294,-0.090488,-0.062443,-0.096493,-0.084902,-0.135516,-0.113903,...,-0.055908,-0.092277,-0.104968,-0.101001,-0.047751,-0.088478,-0.133494,-0.057420,-0.107865,6.400000e-06
2010-06-30,-0.016400,0.030117,-0.176122,-0.081295,-0.026569,-0.062031,0.015836,-0.129125,-0.004264,-0.022281,...,-0.026879,-0.093531,-0.067195,-0.036658,0.018169,-0.166667,-0.107703,-0.049248,-0.056070,7.200000e-06
2010-07-31,0.049166,0.025614,0.086644,-0.001725,0.117015,-0.017055,0.036692,0.078986,0.124433,0.085896,...,0.074234,0.142556,0.069351,0.095363,0.037116,0.069288,0.083203,0.064905,0.045733,6.000000e-06
2010-08-31,0.005297,-0.076690,-0.035515,0.063457,-0.118014,-0.022663,-0.064001,0.058868,-0.106855,-0.102876,...,-0.023169,-0.018462,-0.129707,-0.082841,0.016173,-0.058494,-0.150739,-0.020512,-0.009551,5.600000e-06
2010-09-30,0.058776,0.160929,-0.055957,-0.017646,0.152373,0.143116,0.079741,0.258191,0.054176,0.088500,...,0.121470,0.045298,0.039423,0.092317,0.103623,0.246280,0.066454,0.067411,0.045339,6.400000e-06
2010-10-31,-0.017611,0.052248,0.076482,0.102576,0.074425,-0.033597,0.037743,0.052018,-0.013562,0.061617,...,0.071883,0.009747,0.119334,0.049698,-0.003375,0.011343,0.037627,0.012145,0.076064,4.800000e-06


In [108]:
def returns_expost(returns, config):
    # Construct r_ex(horizon, lambda) = lambda * r_ex(horizon) + (1-lambda) * noise, where noise = N(0, sd(r_ex))
    # r_ex(horizon)
    r_ex = returns.rolling(config['horizon']).sum().shift(-1, fill_value=0)
    
    # noise, where noise = N(0, sd(r_ex(horizon))) - shape(r_ex)
    noise = np.random.normal([0] * len(r_ex.std()), r_ex.std(), r_ex.shape)
    
    return (config['lambda'] * r_ex).add((1 - config['lambda']) * noise).dropna()

if config['returns']['mode'] == 'expost':
    return_estimate = returns_expost(returns, config['returns'])
else:
    return_estimate = returns.ewm(alpha=0.1, min_periods=6).mean().shift(1, fill_value=0).dropna()
return_estimate

,ABT,ACN,ADBE,AGN,AIG,ALL,AMGN,AMZN,AXP,BA,...,UNP,UPS,USB,UTX,VZ,WBA,WFC,WMT,XOM,USDOLLAR
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-28,0.047828,0.052659,-0.047778,-0.061479,-0.004340,0.003821,-0.056931,0.057326,0.050520,-0.051909,...,-0.021340,-0.008409,0.034902,0.009794,0.050046,0.031778,-0.001306,0.001455,-0.066010,-2.734696e-06
2010-03-31,0.094250,0.063584,-0.072144,0.058581,-0.167859,0.015033,0.005329,0.149405,0.017955,-0.027403,...,0.152164,0.019027,-0.028927,0.053004,0.015691,0.061127,-0.048681,-0.007388,0.055100,-3.484302e-05
2010-04-30,-0.007199,0.030025,-0.044732,-0.031928,-0.168630,-0.043252,0.069864,0.032764,-0.131543,-0.055657,...,-0.023316,-0.038183,0.005816,-0.039573,-0.034558,-0.012975,-0.041871,-0.013472,-0.146014,2.674978e-05
2010-05-31,0.045106,-0.071543,-0.081569,-0.018600,0.108209,0.018594,0.068646,-0.075323,-0.003164,-0.068278,...,-0.017601,-0.043026,-0.021173,-0.034745,-0.023537,-0.050353,0.006718,-0.029943,-0.040525,-2.684886e-05
2010-06-30,-0.019879,-0.029833,0.009035,-0.011082,-0.052876,-0.042884,0.118345,0.025629,0.041650,-0.048025,...,0.025512,0.008079,0.026398,-0.070342,0.005546,-0.028122,0.001882,-0.046286,-0.020908,1.317595e-05
2010-07-31,0.076716,-0.011100,-0.066471,-0.026453,-0.051008,-0.014487,0.015636,-0.023601,-0.055709,-0.031444,...,0.054437,0.075929,-0.017901,-0.013080,-0.004156,0.033176,0.037351,-0.037883,-0.043869,4.202689e-06
2010-08-31,0.016272,0.109745,0.025679,0.053287,-0.001330,0.013885,-0.018806,0.034507,0.021155,-0.070337,...,-0.015986,0.029610,0.068463,-0.050008,-0.008763,-0.030128,0.087753,0.006162,-0.008755,7.693755e-06
2010-09-30,0.016718,0.018414,0.012217,0.033329,0.031409,-0.013187,-0.058850,0.033476,0.082280,-0.023359,...,-0.034288,0.042916,0.042854,0.064628,0.016109,0.004759,0.062908,-0.019206,0.009530,3.917526e-05
2010-10-31,0.000806,-0.038737,0.017988,0.008401,-0.059032,-0.046375,0.008729,0.095332,0.127919,-0.025748,...,-0.102877,0.022761,-0.080753,-0.050197,0.041623,0.036845,-0.021646,0.060810,0.014557,2.445662e-05


In [124]:
agree_on_sign=np.sign(returns.iloc[:,:-1]) == np.sign(return_estimate.iloc[:,:-1])
print("Return predictions have the right sign %.1f%% of the times"%
      (100*agree_on_sign.sum().sum()/(agree_on_sign.shape[0]*(agree_on_sign.shape[1]-1))))

Return predictions have the right sign 50.8% of the times


In [129]:
volume_estimate=volumes.ewm(alpha=0.1, min_periods=6).mean().shift(1).dropna()
volume_estimate.to_hdf(datadir+'rex_model.h5', 'volume_estimate')
sigma_estimate = sigmas.ewm(alpha=0.1, min_periods=6).mean().shift(1).dropna()
sigma_estimate.to_hdf(datadir+'rex_model.h5', 'sigma_estimate')
return_estimate.to_hdf(datadir+'rex_model.h5', 'return_estimate')

HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 509, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1400, in H5F__open
    unable to open file
  File "H5Fint.c", line 1615, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1640, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 941, in H5FD_sec2_lock
    unable to lock file, errno = 35, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file '../../data/rex_model.h5'

In [120]:
sigma_estimate.shape

(102, 81)

# Factor Risk Model

In [65]:
if config['risk'] == 'FF':
    from sklearn import linear_model

    start_t="2016-04-01"
    end_t="2018-12-31"

    first_days_biweekly=\
        pd.date_range(start=returns.index[next(i for (i,el) in enumerate(returns.index >= start_t) if el)-1],
                                     end=returns.index[-1], freq='SMS')

    # Use multi linear regression to obtain factor loadings. Then factor covariance and stock idiosyncratic variances
    exposures, factor_sigma, idyos = {}, {}, {}

    # Every first day in each biweekly period
    for day in first_days_biweekly:
        print('Running for {}'.format(day.strftime('%Y %b %d')))

        # Grab asset returns for preceding 90 days
        used_returns = returns.loc[(returns.index < day)&
                            (returns.index >= day-pd.Timedelta("90 days"))]
        used_ff_returns = ff_returns.loc[ff_returns.index.isin(used_returns.index)].iloc[:, :-1]

        # Multi linear regression to extract factor loadings
        mlr = linear_model.LinearRegression()
        mlr.fit(used_ff_returns, used_returns)
        mlr.predict(used_ff_returns)

        # Factor covariance - on EWMA of FF returns
        factor_sigma[day] = used_ff_returns.ewm(alpha=0.1, min_periods=60).cov().iloc[-5:].droplevel(0).fillna(0)
        # Exposures - factor loadings obtained from multi linear regression coefficients of stock on FF factors
        exposures[day] = pd.DataFrame(data=mlr.coef_,index=returns.columns).fillna(0)
        # Stock idiosyncratic variances - stock var minus FF var, ensure >=0
        idyos[day] = pd.Series(np.diag(used_returns.ewm(alpha=0.1, min_periods=60).cov().iloc[-100:].values -
                                        exposures[day].values@factor_sigma[day].values@exposures[day].values.T),
                               index=returns.columns).fillna(method='ffill')
        idyos[day][idyos[day] < 0] = 0

    pd.concat(factor_sigma.values(), axis=0, keys=factor_sigma.keys()).to_hdf(datadir+'rex_model.h5', 'factor_sigma')
    pd.concat(exposures.values(), axis=0, keys=exposures.keys()).to_hdf(datadir+'rex_model.h5', 'exposures')
    pd.DataFrame(idyos).T.to_hdf(datadir+'rex_model.h5', 'idyos')

    # load with
    # store = pd.HDFStore('ff_model.h5')
    # store.exposures, etc.

Running for 2016 Apr 01
Running for 2016 Apr 15
Running for 2016 May 01
Running for 2016 May 15
Running for 2016 Jun 01
Running for 2016 Jun 15
Running for 2016 Jul 01
Running for 2016 Jul 15
Running for 2016 Aug 01
Running for 2016 Aug 15
Running for 2016 Sep 01
Running for 2016 Sep 15
Running for 2016 Oct 01
Running for 2016 Oct 15
Running for 2016 Nov 01
Running for 2016 Nov 15
Running for 2016 Dec 01
Running for 2016 Dec 15
Running for 2017 Jan 01
Running for 2017 Jan 15
Running for 2017 Feb 01
Running for 2017 Feb 15
Running for 2017 Mar 01
Running for 2017 Mar 15
Running for 2017 Apr 01
Running for 2017 Apr 15
Running for 2017 May 01
Running for 2017 May 15
Running for 2017 Jun 01
Running for 2017 Jun 15
Running for 2017 Jul 01
Running for 2017 Jul 15
Running for 2017 Aug 01
Running for 2017 Aug 15
Running for 2017 Sep 01
Running for 2017 Sep 15
Running for 2017 Oct 01
Running for 2017 Oct 15
Running for 2017 Nov 01
Running for 2017 Nov 15
Running for 2017 Dec 01
Running for 2017